## In this notebook, we will import the data we've cleaned from the lending_data_load_and_transform.py script.

## Using the cleaned data, we will perform various data visualizations and examine the data for trends and various relatnioships between the various dimensions and factors. I will denote these relationships using Markdown text and do some scatter plots and regression analysis as well to confirm some of these relationships among the data.

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

import matplotlib

import seaborn as sns

import gc

import random

import statsmodels.api as sm

import statsmodels.formula.api as smf

# specify name of cSV file
lending_data_csv_file_name = 'approved_data_2016_2018.csv'


# initialize empty df to concatenate and store each of the data chunks after importing
df = pd.DataFrame()

# define chunk size
chunksize = 100000

# set data types for columns that we know will match (int, category, etc.), to save on file load times
dtypes = {'id':pd.np.int16,
          'pub_rec':pd.np.int8,
          'fico_range_high':pd.np.int32,
          'fico_range_hlow':pd.np.int8,
          'loan_status':'category',
          'grade':'category',
          'home_ownership':'category',
          'acc_now_delinq':'category',
          'sub_grade':'category',
          'application_type':'category',

          'acc_now_delinq':pd.np.int8
          }

for chunk in pd.read_csv(lending_data_csv_file_name, 
                         chunksize=chunksize, 
                         dtype = dtypes,
                         # use only first 50 columns and skip 1st (0th) column since it is unnamed and skip last n columns since they are mostly null and seemingly irrelevant
                         usecols=range(1,50)):
    df = pd.concat([df, chunk], ignore_index=True)
    # delete each chunk after conatenating the data to the df dataframe
    del chunk; gc.collect()

df.head()
    

d:\Coding and Code projects\Python\craigslist_data_proj\CraigslistWebScraper\craigslist_venv\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
d:\Coding and Code projects\Python\craigslist_data_proj\CraigslistWebScraper\craigslist_venv\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
d:\Coding and Code projects\Python\craigslist_data_proj\CraigslistWebScraper\craigslist_venv\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
d:\Coding and Code projects\Python\craigslist_data_proj\CraigslistWebScraper\craigslist_venv\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: The pandas.np module is deprecated and

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,disbursement_method,debt_settlement_flag
0,13693,5000.0,5000.0,5000.0,36 months,20.39,186.82,D,D4,General Manager,...,1.0,Individual,NaN,NaN,NaN,0,1.0,0.0,Cash,N
1,23769,15000.0,15000.0,15000.0,36 months,9.92,483.45,B,B2,IT Director,...,1.0,Individual,NaN,NaN,NaN,0,0.0,0.0,Cash,N
2,14398,11200.0,11200.0,11200.0,60 months,30.79,367.82,G,G1,Client services,...,1.0,Joint App,81000.0,31.94,Not Verified,0,0.0,0.0,Cash,N
3,21876,25000.0,25000.0,25000.0,60 months,21.85,688.35,D,D5,Asphalt Supervisor,...,1.0,Individual,NaN,NaN,NaN,0,0.0,0.0,Cash,N
4,15138,3000.0,3000.0,3000.0,36 months,7.34,93.10,A,A4,Scale Technician,...,1.0,Individual,NaN,NaN,NaN,0,0.0,0.0,Cash,N


## What are the various loan_status categories--which ones would consistute Good vs Bad performance loans?

## NB: see Lending Club's definitions of these categories: <https://www.lendingclub.com/help/investing-faq/what-do-the-different-note-statuses-mean>
### Examine loan status categories and counts distribution--using .value_counts()--so we can see 

In [2]:
# show value counts distribution of loan status categories
print(f"Loan Status categories counts (distributions) of the loans data:\n{df['loan_status'].value_counts()}")

Loan Status categories counts (distributions) of the loans data:
Current               823093
Fully Paid            402449
Charged Off           116257
Late (31-120 days)     19771
In Grace Period         7623
Late (16-30 days)       3997
Default                   38
Name: loan_status, dtype: int64


## Notice there are 7 loan status categories.

## Per the Lending Club descriptions, only 2 categories can reasonably constitute good performance loans: "Current" and "Fully Paid Off"

## The other 5 categories comprise "Bad" performance loans, albeit to differing degrees

## Next, let's examine the distibutions, categories, and/or ranges for a few borrower-related columns that might correlate with and impact loan performance to some extent:

### Employment length (in years):

In [4]:
print(f"Employment length value counts:")
df.emp_length.value_counts()

Employment length value counts:


10+ years    456411
2 years      124804
< 1 year     119380
3 years      110725
1 year        91308
4 years       84075
5 years       83989
6 years       59676
7 years       48096
8 years       47957
9 years       44735
Name: emp_length, dtype: int64

## Check the dataset for which 'delinquent'  metrics are in this dataset:

In [5]:
# check if any columns exist in the dataset with column name containing the substring 'delinq'
print("Columns in the dataset that contain the substring 'delinq',\nreferring to delinquent account metrics:")
[col for col in df.columns if 'delinq' in col]

Columns in the dataset that contain the substring 'delinq',
referring to delinquent account metrics:


['delinq_2yrs', 'mths_since_last_delinq', 'acc_now_delinq', 'delinq_amnt']

## Show distribution by loan grades--ie, the letter grades assigned by LC, in which A grades are supposed to be the highest grades, followed by B, C, with the lowest graded loans classified as F and G:

In [9]:
#
print("Distribution of data by loan grades:")
df['grade'].value_counts()

Distribution of data by loan grades:


B    409004
C    404172
A    284820
D    184870
E     64932
F     18751
G      6679
Name: grade, dtype: int64

## Check value counts for the delinq_2_yrs: ie, the number of delinquent accounts (per consumer) over the past 2 years:

In [7]:

print("Value counts of the number of delinquent accounts (per consumer) over the past 2 years")
df.delinq_2yrs.value_counts()

Value counts of the number of delinquent accounts (per consumer) over the past 2 years


0.0     1122097
1.0      168122
2.0       47736
3.0       17563
4.0        7852
5.0        3888
6.0        2246
7.0        1278
8.0         762
9.0         534
10.0        364
11.0        242
12.0        174
13.0        101
14.0         75
15.0         59
16.0         38
17.0         20
18.0         19
20.0         15
19.0         15
21.0         10
24.0          3
22.0          2
25.0          2
23.0          2
30.0          1
26.0          1
36.0          1
58.0          1
42.0          1
35.0          1
32.0          1
28.0          1
29.0          1
Name: delinq_2yrs, dtype: int64

## Check for income columns in the dataset, and show value counts for one of them:

In [31]:
# check if any columns exist in the dataset with column name containing the substring 'inc'
[col for col in df.columns if 'inc' in col]


['annual_inc',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'annual_inc_joint']

In [37]:
# show annual income value counts:
print(f"Annual income distribution:\n\n{df['annual_inc'].value_counts()}")

Annual income distribution:

60000.0     52906
50000.0     45776
65000.0     39403
70000.0     37957
80000.0     37103
            ...  
42922.0         1
43087.0         1
101448.0        1
158715.0        1
180792.0        1
Name: annual_inc, Length: 63131, dtype: int64


In [33]:
# distribution of loans by length of loan (term):
print(f"Term of loan (ie, length of loan):\n{df['term'].value_counts()}")

Term of loan (ie, length of loan):
 36 months    988585
 60 months    384643
Name: term, dtype: int64


In [24]:
df['open_acc'].value_counts()

9.0     115469
8.0     112132
10.0    111413
7.0     104930
11.0    102831
         ...  
81.0         1
77.0         1
91.0         1
93.0         1
97.0         1
Name: open_acc, Length: 88, dtype: int64

## Show the distribution by loan title and purposes--ie, what the given loan was used to pay for:

In [28]:
df['title'].value_counts()

Debt consolidation           739286
Credit card refinancing      305358
Home improvement              97324
Other                         95822
Major purchase                32789
Medical expenses              18714
Car financing                 14961
Business                      14076
Vacation                      10638
Home buying                   10349
Moving and relocation          9879
Green loan                      857
Credit Card Consolidation         1
Learning and training             1
Name: title, dtype: int64

In [29]:
df['purpose'].value_counts()

debt_consolidation    753624
credit_card           310777
home_improvement       98623
other                  96544
major_purchase         33168
medical                18947
car                    15149
small_business         14311
vacation               10789
house                  10429
moving                  9988
renewable_energy         870
wedding                    8
educational                1
Name: purpose, dtype: int64

In [ ]:
##

### Data transformations: 

### Create a loan performance categorical column that will be "Good" for good loan status categories, and "Bad" to denote bad loan status categoties such as "Charged off" using np.select()

### Then, create indicator variables based on the loan performance categories to denote loan performance numerically: 

In [8]:
## create a loan performance categorical column
import numpy as np
def create_loan_performance_caterigcal_col(df, col):
    # specify the various conditions for np.selecT() aking to a SQL CASE WHERE statement's arguments
    conditions  = [ df[col] =="Current", 
                   df[col]== "Fully paid",
                   df[col]== "In Grace Period",
                   df[col]== "Late (16-30 days)",
                   df[col]== "Late (31-120 days)",
                   df[col]== "Default",
                   df[col]== "Charged off",
                   ]
    # specify the possible outcomes--in this case, "Good" for good loan performance, and "Bad" for bad performance
    choicelist  = [ "Good", 'Good', "Bad", "Bad", "Bad", "Bad", "Bad"]
    
    # return new categorical column using the specified conditions, choicelist arguments, and a separate default option as numpy NaNs in case any rows in the column do not match any of the specified conditions 
    return np.select(condlist  = conditions,
                     choicelist=choicelist,
                     default = np.nan
    )


# craete loan_performance categorical column
df['loan_performance'] = create_loan_performance_caterigcal_col(df, 'loan_status')

# sanity check
print(f"Sanity check on the loan performance categorical col:\n{df['loan_performance'].value_counts()}")

MemoryError: Unable to allocate 168. MiB for an array with shape (1373228,) and data type <U32

In [ ]:
# create funciton based on get_dummies() method and specify the column name as its prefix
def get_indicator_vars(df, col):
    return pd.get_dummies(df[col], prefix=col)


# create indicator variables of loan performance (status as  Good vs Bad) using np.select()
get_indicator_vars(df, 'loan_performance')

In [ ]:
# next, create LC Loan Grade indicator variables
get_indicator_vars(df, 'grade')

In [ ]:
# next, create employment length indicators
get_indicator_vars('emp_length')

In [ ]:
# next, create home ownership indicators
get_indicator_vars('home_ownership')

### Next, let's run a quick logit regression analysis:

#### Outcome variable is the "Good" loan performance indicator variable--ie, 1's represent "Good" loan performance, 0's represent "Bad" performance
 
#### Explanatory variables will include the following: a) the full set of loan grade indicators, excepting loan grade "C" as a reference group; b) employment length indicators, excepting "5 years" as a reference group; c) int_rate; d) home_ownership  indicators, excepting "Owner" as a reference group; e)revol_util; f) dti; 


## Examining trends in the loan data

###  NB: There are a few basic considerations regarding the dataset structure before examining trends: 

### a) The id column is q unique column that refeers to the ID for a loan listing. Since each ID is unique, we can refer to this column to track loan volume over time. 

### Also: b) issue_d is the relevant datetime column of interest here since this reflects the date on which the loan was originally funded. The dates in this dataset range from Januar 2016 to December 2018.

In [ ]:
# boxplot of loan data by year and month


In [ ]:
# calculate summary statistics by year

# split dataset by year


# summary stats by year
df['loan_data'].describe()

In [ ]:
# # examine for seasonality in loan origination


In [ ]:
## loan data by 

### Examining trends in loan data

#### Interpretation & Analysis:


## Examining Loan grade in terms of loan performnance

## The metric of interest to convey loan performance is given by loan_status. This column has 1 of 2 outcomes: good or bad loan, respectively.

## So to examine whether there is a relationship between loan performance and loan status, we can examine a scatterplot of these 2 columns. 

## But first, let's examine the overall distribution of the loan data in terms of loan status (performance):

In [ ]:
# distribution of the data by loan status

# histogram 

In [ ]:
# What metric do we use to measure loan performance?

# use loan_status since this reflects whether a loan has been fully paid off, is current, or is late or charged off (defaulted) in some way

In [ ]:
# group the data by loan grade category, and then compare summary stats by loan performance

df_loan_grade = df.groupby('loan_grade')

print(f"Summary statistics by loan grade:\n\n{df_loan_grade.describe()}")

In [ ]:
# scatter plot of loan grade vs loan performance

def seaborn_scatterplot(y_var, x_var, hue):
    return sns.


y_var, x_var = 'loan_status', 'loan_grade' 

seaborn_scatterplot()


In [ ]:
# let's also examine sub_grade vs loan_status-- sub_grade includes sub categories of each grade--e.g., A1, A2, A3 for grade  A, etc.



In [ ]:
# regression analysis of loan grade vs loan performance

## Examine what other factors seem to affect loan peformance 

## Plot correlation coefficient matrix to examine the extent to which different factors are correlated with loan performance



In [ ]:
# examine fico_range_low and fico_range_high, respectively compared with loan_status

# rico_range_low vs loan_status scatter plot

In [ ]:
# scatter plot of  home_ownership vs loan_status

In [ ]:
# scatter plot of emp_length vs loan_status


In [ ]:
# scatter plot of installment vs loan_status


In [ ]:
# trends in pymnt_plan over time

In [ ]:
# scattter plot of  purpose vs loan status


In [ ]:
# scattter plot of 	title vs loan status

In [ ]:
#   scattter plot of 	delinq_amnt vs loan status

# NB: a priori, we expect higher delinquincy amounts should be negatively correlated with loan peformance--ie, higher extent of 'Bad' loan status


In [ ]:
## plot of verification_status vs loan status





In [ ]:
## plot of verified_status_joint vs loan status


In [ ]:
# scatter plot of annual_inc vs loan status


In [ ]:
# scatter plot of individual vs joint loan payers vs loan status

### Examine any other trends or important characteristics of the data



In [ ]:
### Taking note of summary statistics and characteristics of the loans data

### Distribution of loans data by loan purpose (i.e., what the loan was used to pay for)
df['purpose'].value_counts()


In [ ]:
# distribution of data by loan_amnt--ie, dollar value of the loan when originally applied for

df['loan_amnt'].value_counts()


In [ ]:
# distribution of data by loan_amnt

df['purpose'].value_counts()


In [ ]:
# distribution of data by addr_state--ie, the state the borrower applied from
df['addr_state'].value_counts()

In [ ]:
# distribution of data by int_rate
df['int_rate'].value_counts()

In [ ]:
# int_rate trends over time

# lineplot of int_rate
lineplot()
df['int_rate']

In [ ]:
# finally, let's examine the distribution of interest rates by loan status to see whether interest rates do differ in that respect

# we can ecompare these distributions by taking boxplots and using loan status as a hue:

def boxplot_with_hue(df, col_for_boxplot, hue_col):
    return sns.boxplot()



boxplot_with_hue(df, 'int_rate', 'laon_status')


## Logit regression analysis:

In [ ]:
 
# defining the outcome and independent variables 
Xtrain = df[[ 'grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G' ]] 
# use Good loan performance indicator column as outcome var
ytrain = df[['loan_performance_Good']] 

# building the model and fitting the data 
logit_reg = sm.Logit(ytrain, Xtrain).fit() 

In [ ]:
# show logit regression results
print(logit_reg.summary()) 